In [19]:
import os
import time
import gnupg
import keyring
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from binance.spot import Spot
from scipy.optimize import minimize
import warnings
warnings.filterwarnings("ignore")
pd.set_option('display.float_format', lambda x: '%.4f' % x)

In [2]:
gpg = gnupg.GPG()
encrypted_file_path = os.path.expanduser('~/Repositorios/binance_trader/.env.gpg')
with open(encrypted_file_path, 'rb') as file:
    datos = gpg.decrypt_file(file, passphrase=keyring.get_password("GPG_Passphrase", "gpg_python"))
if datos.ok:
    env_vars = dict(line.decode('utf-8').split('=', 1) for line in datos.data.splitlines())
    api_key = env_vars.get('API_KEY')
    secret_key = env_vars.get('SECRET_KEY')
# Inicializa el cliente de Binance
client = Spot(api_key, secret_key)

In [41]:
def get_historical_data(symbols, interval='1w'):
    data_frames = {}
    
    for symbol in symbols:
        start_time = 0
        data = []
        
        while True:
            # Obtener datos de klines
            klines = client.klines(symbol=symbol, interval=interval, limit=1000, startTime=start_time)
            # Si no hay más datos, salir del bucle
            if not klines:
                break
            for kline in klines:
                data.append({
                    'open_time': kline[0],
                    'open': float(kline[1]),
                    'high': float(kline[2]),
                    'low': float(kline[3]),
                    'close': float(kline[4]),
                    'volume': float(kline[5]),
                    'quote_asset_volume': float(kline[7]),
                    'number_of_trades': kline[8],
                    'taker_buy_base_asset_volume': float(kline[9]),
                    'taker_buy_quote_asset_volume': float(kline[10])
                })
            # Actualizar start_time para la próxima solicitud
            start_time = klines[-1][0] + 1
        data_frames[symbol] = pd.DataFrame(data)
    # Crear el dataframe multinivel
    df_list = []
    for symbol, df in data_frames.items():
        df = df.set_index('open_time')
        df.index = pd.to_datetime(df.index, unit='ms')
        df.columns = pd.MultiIndex.from_product([[symbol], df.columns])
        df_list.append(df)
    final_df = pd.concat(df_list, axis=1)
    return final_df
# Descarga de datos de activos
# Obtén información sobre el intercambio
exchange_info = client.exchange_info()
# Filtra los pares de trading contra USDT
symbols = ['HBARUSDT', 'XRPUSDT']
df = get_historical_data(symbols).dropna()


In [72]:
# Cálculo de la frontera eficiente
def calculate_efficient_frontier(df_returns):
    def portfolio_volatility(weights, cov_matrix):
        return np.sqrt(np.dot(weights.T, np.dot(cov_matrix, weights)))
    def portfolio_return(weights, expected_returns):
        return np.dot(weights, expected_returns)
    expected_returns = df_returns.mean()
    cov_matrix = df_returns.cov()
    symbols = df_returns.columns.to_list()
    # Restricciones: la suma de los pesos debe ser 1
    constraints = ({'type': 'eq', 'fun': lambda weights: np.sum(weights) - 1})
    # Límites: los pesos deben estar entre 0 y 1
    bounds = tuple((0, 1) for _ in range(len(symbols)))
    # Optimizar la cartera para diferentes niveles de rendimiento esperado
    target_returns = np.linspace(expected_returns.min(), expected_returns.max(), 50)
    efficient_portfolios = []
    sharpe_ratios = []
    for target in target_returns:
        constraints = ({'type': 'eq', 'fun': lambda weights: np.sum(weights) - 1},
                    {'type': 'eq', 'fun': lambda weights: np.dot(weights, expected_returns) - target})
        result = minimize(portfolio_volatility, len(symbols) * [1. / len(symbols),], args=(cov_matrix),
                        method='SLSQP', bounds=bounds, constraints=constraints)
        if result.success:
            weights = result.x
            ret = portfolio_return(weights, expected_returns)
            vol = portfolio_volatility(weights, cov_matrix)
            sharpe_ratios.append(ret / vol)
            efficient_portfolios.append(result.x)
    # Convertir resultados a un DataFrame para análisis
    df_efficient_frontier = pd.DataFrame(efficient_portfolios, columns=symbols, index=sharpe_ratios)
    return df_efficient_frontier.loc[df_efficient_frontier.index.max()]

# Obtener datos históricos y calcular la frontera eficiente
df_returns = df.xs('close', level=1, axis=1).pct_change().dropna()
calculate_efficient_frontier(df_returns)

HBARUSDT   0.4694
XRPUSDT    0.5306
Name: 0.1078528610823511, dtype: float64